In [1]:
#Importing Data from the CSV file
%matplotlib inline
import string
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle

f_train = open("training.1600000.processed.noemoticon.csv", "r", encoding="ISO-8859-1")
X_train = f_train.readlines()
f_test = open("testdata.manual.2009.06.14.csv", "r")
X_test = f_test.readlines()

#Creating the List with just the tweets and finding the number of positive and negative classes. (TRAINING) 
#class_0 = number of classes with label = 0
#class_4 = number of classes with label = 4

class_0=class_4=0
tweets_train =[]
for x in X_train:
    a = x.split('","')
    if (a[0] == '"0'): class_0+=1
    else: class_4+=1
    tweets_train.append('%s"|"%s' % (a[0],a[-1]))

#Creating the List with just the tweets (TEST DATA)
tweets_test =[]
for x in X_test:
    a = x.split('","')
    tweets_test.append('%s"|"%s' % (a[0], a[-1]))
    
print("The length of the training set is = ", len(X_train))
print("The number of classes (label=4) =", class_4)
print("The number of classes (label=0) =", class_0)
print("The length of the test set is = ", len(X_test))

f_test.close()
f_train.close()

m= len(X_train)
phi_0 = class_0/m
phi_4 = class_4/m

The length of the training set is =  1600000
The number of classes (label=4) = 800000
The number of classes (label=0) = 800000
The length of the test set is =  359


In [2]:
import pandas as pd
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectPercentile, chi2


In [3]:
#tweets_train_pd = pd.read_pickle('tweet_train_lemma.pkl')
#tweets_test_pd = pd.read_pickle('tweet_test_lemma.pkl')
train_actual_classes = pd.read_pickle('train_class.pkl')
test_actual_classes = pd.read_pickle('test_class.pkl')

In [4]:
tweets_train_pd = pd.Series(tweets_train)
tweets_train_pd = tweets_train_pd.apply(lambda x: x.split("|")[1])
tweets_test_pd = pd.Series(tweets_test)
tweets_test_pd = tweets_test_pd.apply(lambda x: x.split("|")[1])

In [5]:
train_classes = train_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
test_classes = test_actual_classes.apply(lambda x: 0 if x=='"0"' else 1)
all_classes = train_classes.unique()

In [6]:
def create_minibatch(tweets, classes, batch_size):
    l = len(tweets)
    for i in range(0, l, batch_size):
        X_batch=tweets[i:min(i + batch_size, l)]
        Y_batch=classes[i:min(i + batch_size, l)]
        yield X_batch,Y_batch

In [7]:
model_gauss = GaussianNB()

In [ ]:
#Create minibatches
vec = HashingVectorizer(analyzer="word", stop_words='english',norm='l2')
X = vec.transform(tweets_train_pd)

for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):
    tfidf_train = vec.transform(X_batch)
    model_gauss.partial_fit(tfidf_train.toarray(), Y_batch, classes=all_classes)

In [157]:
X_test = vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

In [162]:
print("The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=1000 is = {}%".format(test_accuracy*100))

The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=1000 is = 55.98885793871866%


In [ ]:
#Create minibatches
import time
start = time.time()
vec = HashingVectorizer(analyzer="word", stop_words='english',norm='l2', n_features=2**18)
#X = vec.transform(tweets_train_pd)

for (X_batch, Y_batch) in create_minibatch(tweets_train_pd, train_classes, 1000):
    tfidf_train = vec.transform(X_batch)
    #print(tfidf_train.shape)
    model_gauss.partial_fit(tfidf_train.toarray(), Y_batch, classes=all_classes)

print("The time taken for training on batch size=10000 using hashing vect only=", time.time() - start)

In [ ]:
X_test = vec.transform(tweets_test_pd)
test_accuracy = model_gauss.score(X_test.toarray(), test_classes)

/home/ayushi/.local/lib/python3.6/site-packages/sklearn/naive_bayes.py:450: RuntimeWarning: invalid value encountered in log
  n_ij = - 0.5 * np.sum(np.log(2. * np.pi * self.sigma_[i, :]))


In [ ]:
print("The Test Accuracy of the GaussianNB model trained with Hashing Vectorizer and batch size=10000 is = {}%".format(test_accuracy*100))

In [12]:
tweets_shuffled = pd.concat([tweets_train_pd, train_classes], axis=1).sample(frac=1)

In [13]:
tweets_train = list(tweets_shuffled[0])
train_classes = list(tweets_shuffled[1])

In [ ]:
vec = TfidfVectorizer(analyzer="word", stop_words='english',sublinear_tf=True, ngram_range=(1,1))
tfidf_train = vec.fit_transform(tweets_train_pd)
tfidf_train_new = SelectPercentile(chi2, percentile=50).fit_transform(tfidf_train, train_classes)
tfidf_train_new